In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
FOLDERNAME = 'OCR/ESRGAN'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

%cd /content/drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/OCR/ESRGAN


In [3]:
!ls

 figures   models	   QA.md       RRDBNet_arch.py	 tmp
 LICENSE   net_interp.py   README.md  'test (1).ipynb'	 transer_RRDB_models.py
 LR	   __pycache__	   results     test.ipynb


In [4]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!huggingface-cli login
# 학교 계정 , Jh ++


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|

In [5]:
!pip install sentencepiece
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task   or "summarization"
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

from langchain import PromptTemplate,  LLMChain
template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

template_korean = """
              다음 텍스트를 삼중 역따옴표로 감싼 간결한 요약을 작성하십시오.
              텍스트의 주요 내용을 다루는 불릿 포인트로 응답을 제공하십시오.
              ```{text}```
              불릿 포인트 요약:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

prompt_korean = PromptTemplate(template=template_korean, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain_korean = LLMChain(prompt=prompt_korean, llm=llm)


# llm_chain.run(text2)


# llm_chain.run_korean(text2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
!pip install easyocr
!pip install Werkzeug
!pip install flask-ngrok
!pip install pyngrok==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4dd6cf9472f30c35deff14b1653018de10af3c61f14e59e2aa4feccabbdf3fe2
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [16]:
!ngrok authtoken 2Xbh1LJ0vJG8OTQoDp3PK48eF70_2x65Acm8WU1WEWNHQgkJV

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import easyocr
from werkzeug.utils import secure_filename
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests

app = Flask(__name__)
run_with_ngrok(app)

def get_ngrok_url():
    from pyngrok import ngrok

    public_url = ngrok.connect(port="5000")  # Replace 5000 with the port your Flask app is using
    print("Ngrok URL:", public_url)
    requests.post("http://k9a708.p.ssafy.io:8081/ocr/url",data={"url":public_url + ""})

get_ngrok_url()  # Get and print the ngrok URL before starting the Flask app

@app.route('/')
def home():
 # Inside a route function, you can access the ngrok URL
    ngrok_url = request.host_url
    print("Ngrok URL:", ngrok_url)

    # Send the ngrok URL to the Spring server
    data = {'url': ngrok_url}

@app.route('/ai/ocr', methods=['POST'])
def ocr():
    device = torch.device('cuda')
    file = request.files['file']
    file.save('./tmp/' + secure_filename(file.filename))
    path = './tmp/' + secure_filename(file.filename)
# 업스케일링 코드
    model_path = 'models/RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth

    model = arch.RRDBNet(3, 3, 64, 23, gc=32)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()
    model = model.to(device)

    base = osp.splitext(osp.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()

    cv2.imwrite('tmp/{:s}_rlt.png'.format(base), output)

# OCR 적용 코드
    reader = easyocr.Reader(['ko', 'en'])
    result = reader.readtext('tmp/{:s}_rlt.png'.format(base), detail = 0)
    # result = reader.readtext(path, detail = 0)

    output = ""

    for x in result:
        output += (x + " ")

    output.replace('    ','  ')
    print(output)
    output = '"""' + output + '"""'

    # a = llm_chain.run(output)
    result = llm_chain.run(output)

    return result

if __name__ == '__main__':
    app.run()


PyngrokNgrokHTTPError: ignored

python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [ ]:


# llm_chain.run(text2)


# llm_chain.run_korean(text2)